<a href="https://colab.research.google.com/github/Exion007/Colab/blob/main/sample_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample Training

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import xgboost as xgb

%matplotlib inline

In [ ]:
df = pd.read_csv("movies.csv")
df.head()

In [ ]:
df.info()

In [ ]:
to_del = ["original_language", "production_companies", "tagline", "popularity", "overview", "release_date"]
df = df.drop(to_del, axis=1)
df = df[df.budget > 0]
df.dropna(inplace = True)

print(df.shape[0])

In [ ]:
def convert_to_list(row):
  row = row[2:-2].replace("'","").split(", ")
  return row

# Convert genres string into list format
df['genres'] = df['genres'].apply(convert_to_list)

def filter_non_float_rows(dataframe, column_name):
    filtered_dataframe = df[df[column_name].apply(lambda x: isinstance(x, float))]
    return filtered_dataframe

df = filter_non_float_rows(df, 'vote_average')

In [ ]:
X = df.drop('vote_average', axis=1)
y = df['vote_average']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
model.fit(X_train_scaled, y_train)

In [ ]:
y_pred = model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f'Root Mean Squared Error: {rmse}')

In [ ]:
new_budget = 10000000  # Replace with the budget of the new movie
new_row = pd.DataFrame({
    'budget': [new_budget],
    'average_votes': [0]  # You might need to set this value based on your dataset
})
new_row_scaled = scaler.transform(new_row)
predicted_rating = model.predict(new_row_scaled)[0]
print(f"Predicted Average Rating: {predicted_rating}")